# Reddit Climate Change - Modeling & Evaluation
Supervision: Prof. Dr. Jan Fabian Ehmke

Group members: Britz Luis, Huber Anja, Krause Felix Elias, Preda Yvonne-Nadine

Time: Summer term 2023 

Data: https://www.kaggle.com/datasets/pavellexyr/the-reddit-climate-change-dataset

In [ ]:
#  Topic detection

# LDA
# https://towardsdatascience.com/end-to-end-topic-modeling-in-python-latent-dirichlet-allocation-lda-35ce4ed6b3e0
# http://rstudio-pubs-static.s3.amazonaws.com/79360_850b2a69980c4488b1db95987a24867a.html

# BERTopic
# https://maartengr.github.io/BERTopic/index.html


In [2]:
# Preparing environment
%pip install bertopic
from bertopic import BERTopic
from sentence_transformers import SentenceTransformer, util
from umap import UMAP
from hdbscan import HDBSCAN
from sklearn.feature_extraction.text import CountVectorizer
from bertopic.vectorizers import ClassTfidfTransformer
import pandas as pd

  Using cached bertopic-0.14.1-py2.py3-none-any.whl (120 kB)
  Using cached hdbscan-0.8.29-cp310-cp310-macosx_10_9_universal2.whl
  Using cached umap_learn-0.5.3-py3-none-any.whl
  Using cached sentence_transformers-2.2.2-py3-none-any.whl
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.3/15.3 MB 36.2 MB/s eta 0:00:0000:0100:01
  Using cached Cython-0.29.34-py2.py3-none-any.whl (988 kB)
  Using cached transformers-4.28.1-py3-none-any.whl (7.0 MB)
  Using cached torch-2.0.0-cp310-none-macosx_11_0_arm64.whl (55.8 MB)
  Using cached torchvision-0.15.1-cp310-cp310-macosx_11_0_arm64.whl (1.4 MB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 36.6 MB/s eta 0:00:00
  Using cached huggingface_hub-0.14.1-py3-none-any.whl (224 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 39.2 MB/s eta 0:00:00a 0:00:01
  Using cached pynndescent-0.5.10-py3-none-any.whl
  Using cached filelock-3.12.0-py3-none-any.whl (10 kB)
  Using cached fsspec-2023.4.0-py3-none-any.whl (153 kB)
     ━━

/opt/homebrew/lib/python3.10/site-packages/umap/distances.py:1063: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit()
/opt/homebrew/lib/python3.10/site-packages/umap/distances.py:1071: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit()
/opt/homebrew/lib/python3.10/site-packages/umap/distances.py:1086: NumbaDeprecationWarning: The 'nopython' keywo

In [5]:
# Load data
clean_posts = pd.read_csv("data/preprocessed_posts.csv")
clean_comments = pd.read_csv("data/preprocessed_comments.csv")

/var/folders/94/kbz524cd32j80_2tm_hwnr780000gn/T/ipykernel_1980/2118955341.py:3: DtypeWarning: Columns (0,1,2,3,4,5,6,7,10,11,12,16) have mixed types. Specify dtype option on import or set low_memory=False.
  clean_comments = pd.read_csv("data/preprocessed_comments.csv")


In [7]:
# count NAs in date column
clean_comments["created_date"].isna().sum()

1160540

In [9]:
# Drop NAs, should be done in preprocessing file, delete it here!
clean_comments = clean_comments.dropna(axis=0)

In [10]:
# check dataset
clean_comments.head(10)

,type,id,subreddit.id,subreddit.name,subreddit.nsfw,created_utc,permalink,body,sentiment,score,body_clean,language,created_date,created_day,created_month,created_year,created_time
0,comment,imlddn9,2qh3l,news,False,1661990368,https://old.reddit.com/r/news/comments/x2cszk/...,Yeah but what the above commenter is saying is...,0.5719,2.0,Yeah but what the above commenter is saying is...,en,2022-08-31,31.0,8.0,2022.0,23:59:28
1,comment,imldbeh,2qn7b,ohio,False,1661990340,https://old.reddit.com/r/Ohio/comments/x2awnp/...,Any comparison of efficiency between solar and...,-0.9877,2.0,Any comparison of efficiency between solar and...,en,2022-08-31,31.0,8.0,2022.0,23:59:00
2,comment,imldado,2qhma,newzealand,False,1661990327,https://old.reddit.com/r/newzealand/comments/x...,I'm honestly waiting for climate change and th...,-0.1143,1.0,I'm honestly waiting for and the impacts of ...,en,2022-08-31,31.0,8.0,2022.0,23:58:47
3,comment,imld6cb,2qi09,sacramento,False,1661990278,https://old.reddit.com/r/Sacramento/comments/x...,Not just Sacramento. It's actually happening a...,0.0000,4.0,Not just Sacramento. It's actually happening a...,en,2022-08-31,31.0,8.0,2022.0,23:57:58
4,comment,imld0kj,2qh1i,askreddit,False,1661990206,https://old.reddit.com/r/AskReddit/comments/x2...,I think climate change tends to get some peopl...,0.6634,1.0,I think tends to get some people riled up. \...,en,2022-08-31,31.0,8.0,2022.0,23:56:46
5,comment,imlctri,3l2gt,walkaway,False,1661990120,https://old.reddit.com/r/walkaway/comments/x2m...,Naaa how could anyone be mad at a face like th...,0.2500,1.0,Naaa how could anyone be mad at a face like th...,en,2022-08-31,31.0,8.0,2022.0,23:55:20
6,comment,imlctc0,2vvve,pastors,False,1661990114,https://old.reddit.com/r/pastors/comments/x2il...,Can i suggest maybe honing in on LGBTQ?\n\nIt'...,0.9779,2.0,Can i suggest maybe honing in on LGBTQ?\n\nIt'...,en,2022-08-31,31.0,8.0,2022.0,23:55:14
7,comment,imlcpab,2qh1i,askreddit,False,1661990065,https://old.reddit.com/r/AskReddit/comments/x2...,They need to change laws so it's more worth se...,0.4690,2.0,They need to laws so it's more worth selling ...,en,2022-08-31,31.0,8.0,2022.0,23:54:25
8,comment,imlcm07,2sa17,hudsonvalley,False,1661990023,https://old.reddit.com/r/hudsonvalley/comments...,Just thought I would share the climatological ...,0.7367,3.0,Just thought I would share the climatological ...,en,2022-08-31,31.0,8.0,2022.0,23:53:43
9,comment,imlcln1,2t7no,futurology,False,1661990019,https://old.reddit.com/r/Futurology/comments/x...,Blaming environmentalists for the failure of n...,0.3094,4.0,Blaming environmentalists for the failure of n...,en,2022-08-31,31.0,8.0,2022.0,23:53:39


# Modeling Topic Clusters

In [15]:
# Create a subset of posts because otherwise it's to much data
subset_comments = clean_comments[(clean_comments['created_year'] == 2010)]

In [16]:
# Create one array of all titles, to feed it into BERT
docs = subset_comments.body_clean.values

In [23]:
# BERT stepwise
# Step 1 - Extract embeddings
embedding_model = SentenceTransformer("all-MiniLM-L6-v2")

# Step 2 - Reduce dimensionality
umap_model = UMAP(n_neighbors=15, n_components=5, min_dist=0.0, metric='cosine')

# Step 3 - Cluster reduced embeddings
hdbscan_model = HDBSCAN(min_cluster_size=15, metric='euclidean', cluster_selection_method='eom', prediction_data=True)

# Step 4 - Tokenize topics
vectorizer_model = CountVectorizer(stop_words="english")

# Step 5 - Create topic representation
ctfidf_model = ClassTfidfTransformer()

# All steps together
topic_model = BERTopic(
  embedding_model=embedding_model,    # Step 1 - Extract embeddings
  umap_model=umap_model,              # Step 2 - Reduce dimensionality
  hdbscan_model=hdbscan_model,        # Step 3 - Cluster reduced embeddings
  vectorizer_model=vectorizer_model,  # Step 4 - Tokenize topics
  ctfidf_model=ctfidf_model,          # Step 5 - Extract topic words
  calculate_probabilities=False,      # Raises speed
  min_topic_size = 300,               # Reduces number of topics
  nr_topics="auto",                    # Reduces number of topics
  verbose=True
)

topics, probs = topic_model.fit_transform(docs)
topic_model.get_topic_info()

Batches: 100%|██████████| 500/500 [04:10<00:00,  2.00it/s]
2023-05-03 10:44:09,965 - BERTopic - Transformed documents to Embeddings
2023-05-03 10:44:40,232 - BERTopic - Reduced dimensionality
2023-05-03 10:44:40,674 - BERTopic - Clustered reduced embeddings
2023-05-03 10:44:45,574 - BERTopic - Reduced number of topics from 106 to 74


,Topic,Count,Name
0,-1,8554,-1_people_gt_global_just
1,0,1544,0_science_people_gt_don
2,1,1154,1_weather_warming_global_snow
3,2,505,2_temperature_atmosphere_water_vapor
4,3,315,3_obama_rudd_labor_party
...,...,...,...
69,68,16,68_grassroots_poptart_article_typety
70,69,16,69_causation_correlation_does_equal
71,70,16,70_economics_economists_nafta_maths
72,71,15,71_ica_water_peru_aquifer


In [24]:
# Get specific topic
topic_model.get_topic(0)

[('science', 0.01219470217097759),
 ('people', 0.010584660869951001),
 ('gt', 0.009517132731414605),
 ('don', 0.0086000351186167),
 ('scientific', 0.008597060961067942),
 ('global', 0.00822324026848448),
 ('think', 0.00787297838775973),
 ('scientists', 0.007625308000465114),
 ('evolution', 0.007521849591286694),
 ('warming', 0.007371242329106124)]

In [27]:
# Store topic info in dataframe
doc_info = topic_model.get_document_info(docs)

In [28]:
# Check out document information
doc_info.head()

,Document,Topic,Name,Top_n_words,Probability,Representative_document
0,Industrial output --&gt; Increased atmospheric...,-1,-1_people_gt_global_just,people - gt - global - just - science - don - ...,0.0,False
1,This is true but only because Australia lacks ...,-1,-1_people_gt_global_just,people - gt - global - just - science - don - ...,0.0,False
2,"Please, explain to us the whole concept.\n\nNe...",-1,-1_people_gt_global_just,people - gt - global - just - science - don - ...,0.0,False
3,"It hasn't been ""d"" some political types prefer...",1,1_weather_warming_global_snow,weather - warming - global - snow - ice - temp...,1.0,False
4,"&gt; It's called "" "" or as the right wing has ...",1,1_weather_warming_global_snow,weather - warming - global - snow - ice - temp...,1.0,False
